In [ ]:
# # !pip install utils
# # !pip install python-utils 
# !pip install tensorflow-object-detection-api
# !unzip dataset_model_Utils_w2v.zip

files/folders I uploaded to the server:

- drug_review_w2v.model.zip (need to unzip)
- drugsComTest_raw.tsv
- drugsComTrain_raw.tsv

In [ ]:
!unzip drug_review_w2v.model.zip

Archive:  drug_review_w2v.model.zip
  inflating: drug_review_w2v.model   


In [ ]:
!pip install gensim --upgrade
import gensim
print(gensim.__version__)

     |████████████████████████████████| 24.1 MB 1.7 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


3.6.0


In [ ]:
import pandas as pd
import numpy as np
# import Utils
import tensorflow as tf
import keras
import pickle
import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.utils import class_weight
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.layers.merge import Concatenate
from keras.layers import Dense, Input, GlobalMaxPooling1D, GlobalAveragePooling1D, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding, Reshape
from keras.layers.merge import Concatenate
from keras.models import Model, load_model
# # from keras.utils import to_categorical, plot_model
from tensorflow.keras.utils import to_categorical
# # from keras.backend.tensorflow_backend import set_session
from tensorflow.compat.v1.keras.backend import set_session

import nltk
# nltk.download('stopwords')

from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import unicodedata
import re
import gc

In [ ]:
gpu = tf.config.experimental.list_physical_devices('GPU')[0]
tf.config.experimental.set_memory_growth(gpu, True)

Preparing our dataset

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [ ]:
loaded_train_df = pd.read_csv('drugsComTrain_raw.tsv', sep='\t')
df = loaded_train_df

def get_sentiment(rating):
  if rating < 4.0:
    return 'neg'
  elif rating >= 4.0 and rating <= 7.0:
    return 'neutral'
  else:
    return 'pos'

df['sentiment'] = df['rating'].map(lambda x: get_sentiment(x))

# encode class names to integers
labelencoder = preprocessing.LabelEncoder()
df['sentiment_encoded'] = labelencoder.fit_transform(df['sentiment'])

X_train, X_val, y_train, y_val = train_test_split(df['review'], df['sentiment_encoded'], train_size=0.8, random_state=1)

In [ ]:
loaded_test_df = pd.read_csv('drugsComTest_raw.tsv', sep='\t')
test_df = loaded_test_df[:2000]

test_df['sentiment'] = test_df['rating'].map(lambda x: get_sentiment(x))
labelencoder = preprocessing.LabelEncoder()
test_df['sentiment_encoded'] = labelencoder.fit_transform(test_df['sentiment'])
X_test = test_df['review']
y_test = test_df['sentiment_encoded']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(129037,)
(32260,)
(2000,)


In [ ]:
test_df['sentiment'].value_counts()

pos        1206
neg         446
neutral     348
Name: sentiment, dtype: int64

In [ ]:
df['sentiment'].value_counts()

pos        97410
neg        35063
neutral    28824
Name: sentiment, dtype: int64

In [ ]:
df['review']

0         "It has no side effect, I take it in combinati...
1         "My son is halfway through his fourth week of ...
2         "I used to take another oral contraceptive, wh...
3         "This is my first time using any form of birth...
4         "Suboxone has completely turned my life around...
                                ...                        
161292    "I wrote my first report in Mid-October of 201...
161293    "I was given this in IV before surgey. I immed...
161294    "Limited improvement after 4 months, developed...
161295    "I&#039;ve been on thyroid medication 49 years...
161296    "I&#039;ve had chronic constipation all my adu...
Name: review, Length: 161297, dtype: object

In [ ]:
#%time data_x, data_y = load_data('dataset/Raw/Train.tsv', [1, 3, 4], True, 1)
MAX_NB_WORDS = 20000  # 300000
EMBEDDING_DIM = 100  # 200
MAX_SEQUENCE_LENGTH = 30  # 250

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(df['review'])

In [ ]:
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_dev = tokenizer.texts_to_sequences(X_val)
sequences_test = tokenizer.texts_to_sequences(X_test)

Xtrain = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='pre')
Xdev = pad_sequences(sequences_dev, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='pre')
Xtest = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='pre')
ytrain = to_categorical(y_train.values)
ydev = to_categorical(y_val.values)
ytest = to_categorical(y_test.values)

del sequences_dev, sequences_train, sequences_test
gc.collect()

0

In [ ]:
class Metrics(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.f1s = 0
    def on_epoch_end(self, batch, logs={}):
        predict = np.asarray(self.model.predict(self.validation_data[0])).argmax(axis=1)
        targ = self.validation_data[1].argmax(axis=1)
        #f1 = f1_score(targ, predict, average='micro')
        mtrcs = precision_recall_fscore_support(targ, predict, average='macro')
        print ("\nPrecision: "+str(mtrcs[0])+"\tRecall: "+str(mtrcs[1])+"\tF1: "+str(mtrcs[2]))
        if (self.f1s < mtrcs[2]):
            self.f1s = mtrcs[2]
            #self.model.save('./model/CNN_static_twitterGeneral_Best_EMRS_model_f1_.hdf5')
            #print("Modelo guardado\n")
        return

def build_model(rand_embed, w2v_trainable=False):
    num_words = len(tokenizer.word_index)+1

    if rand_embed:
        embedding_matrix = np.zeros((num_words, EMBEDDING_DIM), dtype=np.float32)
        embedding_layer = Embedding(num_words, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)
    else:
        w2v_model = Word2Vec.load('drug_review_w2v.model')
        embedding_matrix = np.random.uniform(-0.05, 0.05, size=(len(tokenizer.word_index)+1, EMBEDDING_DIM))
        
        for word, i in tokenizer.word_index.items():
            try:
                embedding_vector = w2v_model.wv[word]
            except KeyError:
                embedding_vector = None
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector

        embedding_layer = Embedding(num_words,
                                    EMBEDDING_DIM,
                                    weights=[embedding_matrix],
                                    input_length=MAX_SEQUENCE_LENGTH,
                                    trainable=w2v_trainable)

        # class_W = class_weight.compute_class_weight('balanced', np.unique(np.asarray(df_train.label)), np.asarray(df_train.label))
        # class_weight_dict = dict(enumerate(class_W))


    # ---- Model layers ----
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,))

    embedded_sequences = embedding_layer(sequence_input)
    drop_out_1 = Dropout(0.3, name='drop_out_1')(embedded_sequences)
    conv_1 = Conv1D(64, 2, activation='relu', name='conv_1')(drop_out_1)
    gmp_1 = GlobalMaxPooling1D(name='gmp_1')(conv_1)
    conv_2 = Conv1D(64, 3, activation='relu', name='conv_2')(drop_out_1)
    gmp_2 = GlobalMaxPooling1D(name='gmp_2')(conv_2)
    # conv_3 = Conv1D(64, 5, activation='relu', name='conv_3')(drop_out_1)
    # gmp_3 = GlobalMaxPooling1D(name='gmp_3')(conv_3)

    concat = Concatenate()([gmp_1,gmp_2])#  ,gmp_3])
    drop_out_2 = Dropout(0.3, name='drop_out_2')(concat)
    mp_dense = Dense(128, activation='relu', name='mp_dense')(drop_out_2)
    # preds = Dense(10, activation='softmax', name='preds')(mp_dense)
    preds = Dense(3, activation='softmax', name='preds')(mp_dense)
    Conv_model = Model(sequence_input, preds)

    Conv_model.compile(loss='categorical_crossentropy',
                optimizer='Adam',
                metrics=['accuracy'])

    metrics = Metrics()

    callbacks_list = [metrics]

    return Conv_model, callbacks_list

In [ ]:
BATCH_SIZE = 256
N_EPOCHS = 20

In [ ]:
def evaluate_model(model_obj):
    # Get predictions with validation set
    model_pred = model_obj.predict(Xtest, batch_size=16)
    model_pred_proba = [float(x[1]) for x in tf.nn.softmax(model_pred)]
    model_pred_label = [0 if x[0] > x[1] else 1 for x in tf.nn.softmax(model_pred)]

    model_pred_labels = np.argmax(model_pred, axis=1)
    # Conv_model_rand_embed_pred_labels[:10]

    y_test_labels = np.argmax(ytest, axis=1)
    # y_test_labels[:10]

    print(classification_report(y_test_labels, model_pred_labels, target_names=labelencoder.classes_))

Model with randomly initialized embeddings

In [ ]:
Conv_model_rand_embed, callbacks_list_rand_embed = build_model(rand_embed=True)

Conv_model_rand_embed.fit(Xtrain, ytrain,
      shuffle=True,
      batch_size=BATCH_SIZE,
      epochs=N_EPOCHS,
      validation_data=(Xdev, ydev))

Epoch 1/10
505/505 [==============================] - 20s 18ms/step - loss: 0.7820 - accuracy: 0.6667 - val_loss: 0.6921 - val_accuracy: 0.7084
Epoch 2/10
505/505 [==============================] - 9s 18ms/step - loss: 0.6488 - accuracy: 0.7288 - val_loss: 0.6464 - val_accuracy: 0.7317
Epoch 3/10
505/505 [==============================] - 9s 17ms/step - loss: 0.5760 - accuracy: 0.7645 - val_loss: 0.6338 - val_accuracy: 0.7432
Epoch 4/10
505/505 [==============================] - 9s 18ms/step - loss: 0.5120 - accuracy: 0.7924 - val_loss: 0.6279 - val_accuracy: 0.7476
Epoch 5/10
505/505 [==============================] - 9s 18ms/step - loss: 0.4630 - accuracy: 0.8138 - val_loss: 0.6218 - val_accuracy: 0.7565
Epoch 6/10
505/505 [==============================] - 9s 19ms/step - loss: 0.4220 - accuracy: 0.8317 - val_loss: 0.6170 - val_accuracy: 0.7702
Epoch 7/10
505/505 [==============================] - 9s 18ms/step - loss: 0.3906 - accuracy: 0.8442 - val_loss: 0.6180 - val_accuracy: 0.777

In [ ]:
Conv_model_rand_embed, callbacks_list_rand_embed = build_model(rand_embed=True)

Conv_model_rand_embed.fit(Xtrain, ytrain,
      shuffle=True,
      batch_size=BATCH_SIZE,
      epochs=N_EPOCHS,
      validation_data=(Xdev, ydev))

Epoch 1/20
505/505 [==============================] - 10s 16ms/step - loss: 0.7918 - accuracy: 0.6633 - val_loss: 0.6928 - val_accuracy: 0.7091
Epoch 2/20
505/505 [==============================] - 8s 15ms/step - loss: 0.6636 - accuracy: 0.7223 - val_loss: 0.6579 - val_accuracy: 0.7254
Epoch 3/20
505/505 [==============================] - 8s 16ms/step - loss: 0.5946 - accuracy: 0.7538 - val_loss: 0.6405 - val_accuracy: 0.7363
Epoch 4/20
505/505 [==============================] - 7s 15ms/step - loss: 0.5410 - accuracy: 0.7791 - val_loss: 0.6288 - val_accuracy: 0.7450
Epoch 5/20
505/505 [==============================] - 8s 15ms/step - loss: 0.4948 - accuracy: 0.8001 - val_loss: 0.6284 - val_accuracy: 0.7531
Epoch 6/20
505/505 [==============================] - 8s 16ms/step - loss: 0.4620 - accuracy: 0.8143 - val_loss: 0.6324 - val_accuracy: 0.7599
Epoch 7/20
505/505 [==============================] - 8s 15ms/step - loss: 0.4310 - accuracy: 0.8279 - val_loss: 0.6420 - val_accuracy: 0.756

In [ ]:
evaluate_model(Conv_model_rand_embed)

              precision    recall  f1-score   support

         neg       0.73      0.71      0.72       446
     neutral       0.57      0.54      0.56       348
         pos       0.86      0.88      0.87      1206

    accuracy                           0.78      2000
   macro avg       0.72      0.71      0.71      2000
weighted avg       0.78      0.78      0.78      2000



Model with pretrained w2v embeddings (trainable=False)

In [ ]:
Conv_model_w2v_untrainable, callbacks_list_w2v_untrainable = build_model(rand_embed=False)

Conv_model_w2v_untrainable.fit(Xtrain, ytrain,
      shuffle=True,
      batch_size=BATCH_SIZE,
      epochs=N_EPOCHS,
      validation_data=(Xdev, ydev))

Epoch 1/20
505/505 [==============================] - 6s 11ms/step - loss: 0.9578 - accuracy: 0.5983 - val_loss: 0.8157 - val_accuracy: 0.6541
Epoch 2/20
505/505 [==============================] - 6s 12ms/step - loss: 0.8420 - accuracy: 0.6367 - val_loss: 0.8088 - val_accuracy: 0.6572
Epoch 3/20
505/505 [==============================] - 5s 10ms/step - loss: 0.8225 - accuracy: 0.6471 - val_loss: 0.7835 - val_accuracy: 0.6681
Epoch 4/20
505/505 [==============================] - 5s 10ms/step - loss: 0.8107 - accuracy: 0.6529 - val_loss: 0.7758 - val_accuracy: 0.6720
Epoch 5/20
505/505 [==============================] - 5s 10ms/step - loss: 0.8023 - accuracy: 0.6544 - val_loss: 0.7690 - val_accuracy: 0.6765
Epoch 6/20
505/505 [==============================] - 5s 10ms/step - loss: 0.7969 - accuracy: 0.6581 - val_loss: 0.7626 - val_accuracy: 0.6784
Epoch 7/20
505/505 [==============================] - 5s 10ms/step - loss: 0.7927 - accuracy: 0.6608 - val_loss: 0.7567 - val_accuracy: 0.6786

Model with pretrained w2v embeddings (trainable=True)

In [ ]:
evaluate_model(Conv_model_w2v_untrainable)

              precision    recall  f1-score   support

         neg       0.60      0.61      0.61       446
     neutral       0.54      0.04      0.07       348
         pos       0.73      0.92      0.81      1206

    accuracy                           0.70      2000
   macro avg       0.62      0.52      0.50      2000
weighted avg       0.67      0.70      0.64      2000



In [ ]:
BATCH_SIZE = 256
N_EPOCHS = 50


from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience=2, restore_best_weights=True)

Conv_model_w2v_trainable, callbacks_list_w2v_trainable = build_model(rand_embed=False, w2v_trainable=True)

history = Conv_model_w2v_trainable.fit(Xtrain, ytrain,
      shuffle=True,
      batch_size=BATCH_SIZE,
      epochs=N_EPOCHS,
      callbacks=[early_stopping],
      validation_data=(Xdev, ydev))

Epoch 1/50
505/505 [==============================] - 8s 15ms/step - loss: 0.9858 - accuracy: 0.5918 - val_loss: 0.8139 - val_accuracy: 0.6577
Epoch 2/50
505/505 [==============================] - 8s 15ms/step - loss: 0.8265 - accuracy: 0.6458 - val_loss: 0.7748 - val_accuracy: 0.6729
Epoch 3/50
505/505 [==============================] - 7s 15ms/step - loss: 0.7906 - accuracy: 0.6630 - val_loss: 0.7490 - val_accuracy: 0.6848
Epoch 4/50
505/505 [==============================] - 8s 15ms/step - loss: 0.7665 - accuracy: 0.6767 - val_loss: 0.7446 - val_accuracy: 0.6878
Epoch 5/50
505/505 [==============================] - 8s 16ms/step - loss: 0.7474 - accuracy: 0.6841 - val_loss: 0.7205 - val_accuracy: 0.6985
Epoch 6/50
505/505 [==============================] - 8s 16ms/step - loss: 0.7301 - accuracy: 0.6917 - val_loss: 0.7067 - val_accuracy: 0.7027
Epoch 7/50
505/505 [==============================] - 8s 15ms/step - loss: 0.7139 - accuracy: 0.6998 - val_loss: 0.7005 - val_accuracy: 0.7063

In [ ]:
evaluate_model(Conv_model_w2v_trainable)

              precision    recall  f1-score   support

         neg       0.68      0.67      0.67       446
     neutral       0.43      0.22      0.29       348
         pos       0.79      0.91      0.85      1206

    accuracy                           0.74      2000
   macro avg       0.63      0.60      0.60      2000
weighted avg       0.70      0.74      0.71      2000

